# Web Access Logs Analytics
### Data Engineering Capstone Project

#### Project Summary

----



The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [14]:
# Do all imports and installs here
import pandas as pd
import zipfile
import os
import re
import socket
from datetime import datetime

In [34]:
#TEST REGEX
#ip="157.55.39.125"

#regex="^(\d+).(\d+).(\d+).(\d+)$"

#result = re.search(regex, ip)

#print(result.groups())

('157', '55', '39', '125')


### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [3]:
# Read in the data here
#access_log_path="/home/workspace/data/access.log.zip"
access_log_path_sample="/home/workspace/data/access_log_sample.txt"

#df = pd.read_csv(access_log_path, compression='zip', header=0, sep='\n')
df = pd.read_csv(access_log_path_sample, header=None, sep='\n')

names = ["Raw Web Log"]
df.columns = names

In [4]:
df.head()

,Raw Web Log
0,54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] ...
1,"31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] ""..."
2,"31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] ""..."
3,40.77.167.129 - - [22/Jan/2019:03:56:17 +0330]...
4,"91.99.72.15 - - [22/Jan/2019:03:56:17 +0330] ""..."


In [5]:
def parse_web_logs(input_file_path=""):
    
    access_web_log_full_names = ["client_ip", 
                                 "rfc931",
                                 "username", 
                                 "datetime",
                                 "http_method",
                                 "http_url",
                                 "http_protocol",
                                 "http_status_code", 
                                 "bytes_sent", 
                                 "referrer_url",
                                 "user_agent", 
                                 "cookies"]
    
    combined_regex = '^(\S+) (\S+) (\S+) \[(\w{1,2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2}\s[+\-]\d{4})\] "(\S+)\s?(\S+)?\s?(\S+)?" (\d{3}|-) (\d+|-)\s?"?([^"]*)"?\s?"?([^"]*)?"\s?"?([^"]*)?"?$'
    
    reject_file="web_logs_reject.txt"
    keep = []
    reject = []
    
    try:
        
        with open(f"{input_file_path}", "r") as f:
            while True:
                
                line = f.readline()
                
                if line == "":
                    break
                    
                re_pattern = re.search(combined_regex, line)
                
                if len(re_pattern.groups()) != 12:
                    reject.append(line)
                    
                else:
                    keep.append([re_pattern.group(1),
                                 re_pattern.group(2),
                                 re_pattern.group(3),
                                 re_pattern.group(4),
                                 re_pattern.group(5),
                                 re_pattern.group(6),
                                 re_pattern.group(7),
                                 re_pattern.group(8),
                                 re_pattern.group(9),
                                 re_pattern.group(10),
                                 re_pattern.group(11),
                                 re_pattern.group(12)])
        
        if os.path.exists(f"{reject_file}"):
            os.remove(f"{reject_file}")
            
        with open(f"{reject_file}", "w+") as f:
            for line in reject:
                f.write(reject)
        
        return pd.DataFrame(keep, columns = access_web_log_full_names)
    
    except Exception as e:
        print(e)


In [6]:
df_parsed = parse_web_logs(access_log_path_sample)

In [7]:
df_parsed.head(10)

,client_ip,rfc931,username,datetime,http_method,http_url,http_protocol,http_status_code,bytes_sent,referrer_url,user_agent,cookies
0,54.36.149.41,-,-,22/Jan/2019:03:56:14 +0330,GET,/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C...,HTTP/1.1,200,30577,-,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...,-
1,31.56.96.51,-,-,22/Jan/2019:03:56:16 +0330,GET,/image/60844/productModel/200x200,HTTP/1.1,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,-
2,31.56.96.51,-,-,22/Jan/2019:03:56:16 +0330,GET,/image/61474/productModel/200x200,HTTP/1.1,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,-
3,40.77.167.129,-,-,22/Jan/2019:03:56:17 +0330,GET,/image/14925/productModel/100x100,HTTP/1.1,200,1696,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-
4,91.99.72.15,-,-,22/Jan/2019:03:56:17 +0330,GET,/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%...,HTTP/1.1,200,41483,-,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...,-
5,40.77.167.129,-,-,22/Jan/2019:03:56:17 +0330,GET,/image/23488/productModel/150x150,HTTP/1.1,200,2654,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-
6,40.77.167.129,-,-,22/Jan/2019:03:56:18 +0330,GET,/image/45437/productModel/150x150,HTTP/1.1,200,3688,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-
7,40.77.167.129,-,-,22/Jan/2019:03:56:18 +0330,GET,/image/576/article/100x100,HTTP/1.1,200,14776,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-
8,66.249.66.194,-,-,22/Jan/2019:03:56:18 +0330,GET,"/filter/b41,b665,c150%7C%D8%A8%D8%AE%D8%A7%D8%...",HTTP/1.1,200,34277,-,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...,-
9,40.77.167.129,-,-,22/Jan/2019:03:56:18 +0330,GET,/image/57710/productModel/100x100,HTTP/1.1,200,1695,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-


In [8]:
# Combine with nslookup data for each client IP
# Requires socket module
def getHostName(ip_address=""):
    
    ip_regex="^(\d+).(\d+).(\d+).(\d+)$"
    pattern = re.search(ip_regex, ip_address)
    
    
    if ip_address == "":
        return ""
    elif len(pattern.groups()) == 4:
        try:
            lookup = socket.gethostbyaddr(f"{ip_address}")
            return lookup
        except Exception as e:
            return e
    else:
        return ""
    

In [9]:
df_parsed["host_name"] = df_parsed.client_ip.apply(getHostName)

In [10]:
df_parsed.head(10)

,client_ip,rfc931,username,datetime,http_method,http_url,http_protocol,http_status_code,bytes_sent,referrer_url,user_agent,cookies,host_name
0,54.36.149.41,-,-,22/Jan/2019:03:56:14 +0330,GET,/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C...,HTTP/1.1,200,30577,-,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...,-,"(ip-54-36-149-41.a.ahrefs.com, [], [54.36.149...."
1,31.56.96.51,-,-,22/Jan/2019:03:56:16 +0330,GET,/image/60844/productModel/200x200,HTTP/1.1,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,-,"(31-56-96-51.shatel.ir, [], [31.56.96.51])"
2,31.56.96.51,-,-,22/Jan/2019:03:56:16 +0330,GET,/image/61474/productModel/200x200,HTTP/1.1,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,-,"(31-56-96-51.shatel.ir, [], [31.56.96.51])"
3,40.77.167.129,-,-,22/Jan/2019:03:56:17 +0330,GET,/image/14925/productModel/100x100,HTTP/1.1,200,1696,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-,"(msnbot-40-77-167-129.search.msn.com, [], [40...."
4,91.99.72.15,-,-,22/Jan/2019:03:56:17 +0330,GET,/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%...,HTTP/1.1,200,41483,-,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...,-,"(91.99.72.15.parsonline.net, [], [91.99.72.15])"
5,40.77.167.129,-,-,22/Jan/2019:03:56:17 +0330,GET,/image/23488/productModel/150x150,HTTP/1.1,200,2654,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-,"(msnbot-40-77-167-129.search.msn.com, [], [40...."
6,40.77.167.129,-,-,22/Jan/2019:03:56:18 +0330,GET,/image/45437/productModel/150x150,HTTP/1.1,200,3688,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-,"(msnbot-40-77-167-129.search.msn.com, [], [40...."
7,40.77.167.129,-,-,22/Jan/2019:03:56:18 +0330,GET,/image/576/article/100x100,HTTP/1.1,200,14776,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-,"(msnbot-40-77-167-129.search.msn.com, [], [40...."
8,66.249.66.194,-,-,22/Jan/2019:03:56:18 +0330,GET,"/filter/b41,b665,c150%7C%D8%A8%D8%AE%D8%A7%D8%...",HTTP/1.1,200,34277,-,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...,-,"(crawl-66-249-66-194.googlebot.com, [], [66.24..."
9,40.77.167.129,-,-,22/Jan/2019:03:56:18 +0330,GET,/image/57710/productModel/100x100,HTTP/1.1,200,1695,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-,"(msnbot-40-77-167-129.search.msn.com, [], [40...."


In [11]:
df_parsed[df_parsed["cookies"] != "-"]

,client_ip,rfc931,username,datetime,http_method,http_url,http_protocol,http_status_code,bytes_sent,referrer_url,user_agent,cookies,host_name
647,66.249.83.1,-,-,22/Jan/2019:03:58:26 +0330,GET,/article/606/%D8%B9%D9%84%D8%AA-%D9%88-%D8%AF%...,HTTP/1.1,302,0,-,Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexu...,103.28.132.65,"(google-proxy-66-249-83-1.google.com, [], [66...."
662,66.249.83.5,-,-,22/Jan/2019:03:58:28 +0330,GET,/m/article/606/%D8%B9%D9%84%D8%AA-%D9%88-%D8%A...,HTTP/1.1,200,24561,-,Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexu...,103.28.132.65,"(google-proxy-66-249-83-5.google.com, [], [66...."
670,66.249.83.91,-,-,22/Jan/2019:03:58:29 +0330,GET,/static/images/guarantees/goodShopping.png,HTTP/1.1,200,6496,https://www.zanbil.ir/m/article/606/%D8%B9%D9%...,Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexu...,103.28.132.65,"(google-proxy-66-249-83-91.google.com, [], [66..."
672,66.249.83.93,-,-,22/Jan/2019:03:58:29 +0330,GET,/static/images/guarantees/bestPrice.png,HTTP/1.1,200,7356,https://www.zanbil.ir/m/article/606/%D8%B9%D9%...,Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexu...,103.28.132.65,"(google-proxy-66-249-83-93.google.com, [], [66..."
673,66.249.83.92,-,-,22/Jan/2019:03:58:29 +0330,GET,/static/images/guarantees/fastDelivery.png,HTTP/1.1,200,7713,https://www.zanbil.ir/m/article/606/%D8%B9%D9%...,Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexu...,103.28.132.65,"(google-proxy-66-249-83-92.google.com, [], [66..."
674,66.249.83.91,-,-,22/Jan/2019:03:58:29 +0330,GET,/static/images/guarantees/support.png,HTTP/1.1,200,6454,https://www.zanbil.ir/m/article/606/%D8%B9%D9%...,Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexu...,103.28.132.65,"(google-proxy-66-249-83-91.google.com, [], [66..."
675,66.249.83.92,-,-,22/Jan/2019:03:58:29 +0330,GET,/static/images/guarantees/warranty.png,HTTP/1.1,200,5807,https://www.zanbil.ir/m/article/606/%D8%B9%D9%...,Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexu...,103.28.132.65,"(google-proxy-66-249-83-92.google.com, [], [66..."
678,66.249.83.7,-,-,22/Jan/2019:03:58:29 +0330,GET,/settings/logo,HTTP/1.1,200,4120,https://www.zanbil.ir/m/article/606/%D8%B9%D9%...,Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexu...,103.28.132.65,"(google-proxy-66-249-83-7.google.com, [], [66...."
680,66.249.83.91,-,-,22/Jan/2019:03:58:29 +0330,GET,/image/get?path=/Image/x2l8ghv3.jpg,HTTP/1.1,200,23903,https://www.zanbil.ir/m/article/606/%D8%B9%D9%...,Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexu...,103.28.132.65,"(google-proxy-66-249-83-91.google.com, [], [66..."
685,66.249.83.92,-,-,22/Jan/2019:03:58:30 +0330,GET,/static/images/amp/blog.png,HTTP/1.1,200,3863,https://www.zanbil.ir/m/article/606/%D8%B9%D9%...,Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexu...,103.28.132.65,"(google-proxy-66-249-83-92.google.com, [], [66..."


In [12]:
#goog_organic = df_parsed[df_parsed['referrer_url'].str.contains('google\.com/search')]
df_parsed[df_parsed["referrer_url"] != "-"].head(10)

,client_ip,rfc931,username,datetime,http_method,http_url,http_protocol,http_status_code,bytes_sent,referrer_url,user_agent,cookies,host_name
1,31.56.96.51,-,-,22/Jan/2019:03:56:16 +0330,GET,/image/60844/productModel/200x200,HTTP/1.1,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,-,"(31-56-96-51.shatel.ir, [], [31.56.96.51])"
2,31.56.96.51,-,-,22/Jan/2019:03:56:16 +0330,GET,/image/61474/productModel/200x200,HTTP/1.1,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,-,"(31-56-96-51.shatel.ir, [], [31.56.96.51])"
12,178.253.33.51,-,-,22/Jan/2019:03:56:19 +0330,GET,/m/product/32574/62991/%D9%85%D8%A7%D8%B4%DB%8...,HTTP/1.1,200,20406,https://www.zanbil.ir/m/filter/p5767%2Ct156?na...,Mozilla/5.0 (Linux; Android 5.1; HTC Desire 72...,-,[Errno 1] Unknown host
21,178.253.33.51,-,-,22/Jan/2019:03:56:20 +0330,GET,/settings/logo,HTTP/1.1,200,4120,https://www.zanbil.ir/m/product/32574/62991/%D...,Mozilla/5.0 (Linux; Android 5.1; HTC Desire 72...,-,[Errno 1] Unknown host
23,31.56.96.51,-,-,22/Jan/2019:03:56:20 +0330,GET,/image/60819/productModel/200x200,HTTP/1.1,200,7171,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,-,"(31-56-96-51.shatel.ir, [], [31.56.96.51])"
24,31.56.96.51,-,-,22/Jan/2019:03:56:20 +0330,GET,/image/60847/productModel/200x200,HTTP/1.1,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,-,"(31-56-96-51.shatel.ir, [], [31.56.96.51])"
25,178.253.33.51,-,-,22/Jan/2019:03:56:21 +0330,GET,/image/32574?name=pr465at.jpg&wh=max,HTTP/1.1,200,28825,https://www.zanbil.ir/m/product/32574/62991/%D...,Mozilla/5.0 (Linux; Android 5.1; HTC Desire 72...,-,[Errno 1] Unknown host
26,5.78.198.52,-,-,22/Jan/2019:03:56:21 +0330,GET,/m/product/33978/64784/%DA%AF%D9%88%D8%B4%DB%8...,HTTP/1.1,200,21931,https://www.zanbil.ir/m/browse/cell-phone/%DA%...,Mozilla/5.0 (Linux; Android 8.0.0; SAMSUNG SM-...,-,[Errno 1] Unknown host
27,178.253.33.51,-,-,22/Jan/2019:03:56:21 +0330,GET,/image/32574?name=pr465at3.jpg&wh=max,HTTP/1.1,200,24001,https://www.zanbil.ir/m/product/32574/62991/%D...,Mozilla/5.0 (Linux; Android 5.1; HTC Desire 72...,-,[Errno 1] Unknown host
30,5.78.198.52,-,-,22/Jan/2019:03:56:22 +0330,GET,/settings/logo,HTTP/1.1,200,4120,https://www.zanbil.ir/m/product/33978/64784/%D...,Mozilla/5.0 (Linux; Android 8.0.0; SAMSUNG SM-...,-,[Errno 1] Unknown host


In [17]:
#Convert date 
#type(df_parsed["datetime"][0]) # str

# 22/Jan/2019:03:56:16 +0330
# dd/mon/yyyy:hh:mm:ss z
formatting = "%d/%b/%Y:%H:%M:%S %z"
df_parsed["datetime_fmt"] = df_parsed["datetime"].apply(lambda x: datetime.strptime(x, formatting))


In [18]:
df_parsed.head()

,client_ip,rfc931,username,datetime,http_method,http_url,http_protocol,http_status_code,bytes_sent,referrer_url,user_agent,cookies,host_name,datetime_fmt
0,54.36.149.41,-,-,22/Jan/2019:03:56:14 +0330,GET,/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C...,HTTP/1.1,200,30577,-,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...,-,"(ip-54-36-149-41.a.ahrefs.com, [], [54.36.149....",2019-01-22 03:56:14+03:30
1,31.56.96.51,-,-,22/Jan/2019:03:56:16 +0330,GET,/image/60844/productModel/200x200,HTTP/1.1,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,-,"(31-56-96-51.shatel.ir, [], [31.56.96.51])",2019-01-22 03:56:16+03:30
2,31.56.96.51,-,-,22/Jan/2019:03:56:16 +0330,GET,/image/61474/productModel/200x200,HTTP/1.1,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,-,"(31-56-96-51.shatel.ir, [], [31.56.96.51])",2019-01-22 03:56:16+03:30
3,40.77.167.129,-,-,22/Jan/2019:03:56:17 +0330,GET,/image/14925/productModel/100x100,HTTP/1.1,200,1696,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-,"(msnbot-40-77-167-129.search.msn.com, [], [40....",2019-01-22 03:56:17+03:30
4,91.99.72.15,-,-,22/Jan/2019:03:56:17 +0330,GET,/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%...,HTTP/1.1,200,41483,-,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...,-,"(91.99.72.15.parsonline.net, [], [91.99.72.15])",2019-01-22 03:56:17+03:30


In [19]:
# Order by datetime and then ip

df_parsed_sorted = df_parsed.sort_values(by=["datetime_fmt", "client_ip"])

df_parsed_sorted.head(20)

,client_ip,rfc931,username,datetime,http_method,http_url,http_protocol,http_status_code,bytes_sent,referrer_url,user_agent,cookies,host_name,datetime_fmt
0,54.36.149.41,-,-,22/Jan/2019:03:56:14 +0330,GET,/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C...,HTTP/1.1,200,30577,-,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...,-,"(ip-54-36-149-41.a.ahrefs.com, [], [54.36.149....",2019-01-22 03:56:14+03:30
1,31.56.96.51,-,-,22/Jan/2019:03:56:16 +0330,GET,/image/60844/productModel/200x200,HTTP/1.1,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,-,"(31-56-96-51.shatel.ir, [], [31.56.96.51])",2019-01-22 03:56:16+03:30
2,31.56.96.51,-,-,22/Jan/2019:03:56:16 +0330,GET,/image/61474/productModel/200x200,HTTP/1.1,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,-,"(31-56-96-51.shatel.ir, [], [31.56.96.51])",2019-01-22 03:56:16+03:30
3,40.77.167.129,-,-,22/Jan/2019:03:56:17 +0330,GET,/image/14925/productModel/100x100,HTTP/1.1,200,1696,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-,"(msnbot-40-77-167-129.search.msn.com, [], [40....",2019-01-22 03:56:17+03:30
5,40.77.167.129,-,-,22/Jan/2019:03:56:17 +0330,GET,/image/23488/productModel/150x150,HTTP/1.1,200,2654,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-,"(msnbot-40-77-167-129.search.msn.com, [], [40....",2019-01-22 03:56:17+03:30
4,91.99.72.15,-,-,22/Jan/2019:03:56:17 +0330,GET,/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%...,HTTP/1.1,200,41483,-,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...,-,"(91.99.72.15.parsonline.net, [], [91.99.72.15])",2019-01-22 03:56:17+03:30
10,207.46.13.136,-,-,22/Jan/2019:03:56:18 +0330,GET,/product/10214,HTTP/1.1,200,39677,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-,"(msnbot-207-46-13-136.search.msn.com, [], [207...",2019-01-22 03:56:18+03:30
6,40.77.167.129,-,-,22/Jan/2019:03:56:18 +0330,GET,/image/45437/productModel/150x150,HTTP/1.1,200,3688,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-,"(msnbot-40-77-167-129.search.msn.com, [], [40....",2019-01-22 03:56:18+03:30
7,40.77.167.129,-,-,22/Jan/2019:03:56:18 +0330,GET,/image/576/article/100x100,HTTP/1.1,200,14776,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-,"(msnbot-40-77-167-129.search.msn.com, [], [40....",2019-01-22 03:56:18+03:30
9,40.77.167.129,-,-,22/Jan/2019:03:56:18 +0330,GET,/image/57710/productModel/100x100,HTTP/1.1,200,1695,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,-,"(msnbot-40-77-167-129.search.msn.com, [], [40....",2019-01-22 03:56:18+03:30


In [ ]:
#Sessionize



### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.